# Nettoyage des Données

Les données extraites de l'INSEE contiennent de nombreuses colonnes techniques. On va sélectionner les colonnes pertinentes, renommer les variables et préparer les données pour l'analyse.

## 1. Configuration

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Chemins
PROJECT_ROOT = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(PROJECT_ROOT, 'Economie_Demographie', 'data')
CLEAN_DIR = os.path.join(DATA_DIR, 'clean')
os.makedirs(CLEAN_DIR, exist_ok=True)

print(f"Données brutes : {DATA_DIR}")
print(f"Données clean : {CLEAN_DIR}")

Données brutes : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data
Données clean : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data/clean


## 2. Inventaire des fichiers

In [2]:
fichiers_csv = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.csv')])
print(f"{len(fichiers_csv)} fichiers CSV disponibles :\n")
for f in fichiers_csv:
    size = os.path.getsize(os.path.join(DATA_DIR, f)) / 1024
    print(f"  {f:45} {size:>8.1f} KB")

28 fichiers CSV disponibles :

  FILO2021_DEC_COM.csv                           47680.5 KB
  FILO2021_DEC_PAUVRES_COM.csv                   10178.8 KB
  FILO2021_DISP_COM.csv                          59980.6 KB
  FILO2021_DISP_PAUVRES_COM.csv                  10316.0 KB
  FILO2021_TRDECILES_DEC_COM.csv                  4891.4 KB
  FILO2021_TRDECILES_DISP_COM.csv                 8782.8 KB
  chomage_departements_complet.csv                9874.0 KB
  chomage_regions.csv                             1218.1 KB
  chomage_series.csv                             14556.1 KB
  construction_logements.csv                     21999.2 KB
  emploi_departements.csv                         1068.0 KB
  entreprises_departements.csv                     576.5 KB
  ipc_france.csv                                   331.7 KB
  meta_FILO2021_DEC_COM.csv                       2911.2 KB
  meta_FILO2021_DEC_PAUVRES_COM.csv               2859.0 KB
  meta_FILO2021_DISP_COM.csv                      2939.5 KB
  meta_FI

## 3. Fonctions de nettoyage

Deux types de fichiers INSEE :
- **Séries temporelles** (TIME_PERIOD, OBS_VALUE, IDBANK...) : PIB, IPC, chômage...
- **Données localisées** (CODEGEO, OBS_VALUE, DATA_DATE...) : recensement, emploi, revenus...

In [3]:
def nettoyer_insee(df):
    """Nettoie un DataFrame INSEE de type série temporelle."""
    cols_utiles = ['TIME_PERIOD', 'OBS_VALUE', 'TITLE_FR', 'REF_AREA', 'IDBANK']
    cols_dispo = [c for c in cols_utiles if c in df.columns]
    
    df_clean = df[cols_dispo].copy()
    
    renommage = {
        'TIME_PERIOD': 'periode',
        'OBS_VALUE': 'valeur',
        'TITLE_FR': 'description',
        'REF_AREA': 'zone',
        'IDBANK': 'id_serie'
    }
    df_clean = df_clean.rename(columns={k:v for k,v in renommage.items() if k in df_clean.columns})
    
    # Extraction de l'année (gère les formats string "2020-Q1" et int 1975)
    if 'periode' in df_clean.columns:
        if df_clean['periode'].dtype == 'object':
            df_clean['annee'] = df_clean['periode'].str[:4].astype(int)
        else:
            df_clean['annee'] = df_clean['periode'].astype(int)
    
    if 'valeur' in df_clean.columns:
        df_clean = df_clean.dropna(subset=['valeur'])
    
    return df_clean


def nettoyer_recensement(df, nom_valeur='valeur'):
    """Nettoie un DataFrame INSEE de type recensement/fichier localisé."""
    cols_garder = ['CODEGEO', 'OBS_VALUE', 'DATA_DATE', 'UNIT_label_fr']
    
    # Colonnes descriptives optionnelles
    cols_desc = ['STOCD_label', 'NA17_label', 'NA10_HORS_AZ_label', 
                 'INDICS_FILO_DISP_label', 'UNIT']
    for c in cols_desc:
        if c in df.columns:
            cols_garder.append(c)
    
    cols_dispo = [c for c in cols_garder if c in df.columns]
    df_clean = df[cols_dispo].copy()
    
    renommage = {
        'CODEGEO': 'code_dept',
        'OBS_VALUE': nom_valeur,
        'DATA_DATE': 'annee',
        'UNIT_label_fr': 'unite'
    }
    df_clean = df_clean.rename(columns={k:v for k,v in renommage.items() if k in df_clean.columns})
    
    if nom_valeur in df_clean.columns:
        df_clean = df_clean.dropna(subset=[nom_valeur])
    
    if 'annee' in df_clean.columns:
        df_clean['annee'] = df_clean['annee'].astype(int)
    
    # Suppression des colonnes avec plus de 50% de NaN
    seuil = len(df_clean) * 0.5
    df_clean = df_clean.dropna(axis=1, thresh=seuil)
    
    df_clean = df_clean.drop_duplicates()
    
    return df_clean

---

## 4. PIB France

In [4]:
df = pd.read_csv(os.path.join(DATA_DIR, 'pib_france.csv'))
print(f"Brut : {df.shape}")

df_pib = nettoyer_insee(df)
df_pib['trimestre'] = df_pib['periode'].str[-1].astype(int)

print(f"Nettoyé : {df_pib.shape}")
print(f"Période : {df_pib['periode'].min()} → {df_pib['periode'].max()}")
df_pib.head()

Brut : (20435, 56)
Nettoyé : (20435, 7)
Période : 1949-Q1 → 2025-Q3


,periode,valeur,description,zone,id_serie,annee,trimestre
0,1950-Q2,1.71,Demande intérieure finale - Contribution à l'é...,FE,11794832,1950,2
1,1950-Q3,2.12,Demande intérieure finale - Contribution à l'é...,FE,11794832,1950,3
2,1950-Q4,1.17,Demande intérieure finale - Contribution à l'é...,FE,11794832,1950,4
3,1951-Q1,0.51,Demande intérieure finale - Contribution à l'é...,FE,11794832,1951,1
4,1951-Q2,1.64,Demande intérieure finale - Contribution à l'é...,FE,11794832,1951,2


## 5. IPC (Inflation)

In [5]:
df = pd.read_csv(os.path.join(DATA_DIR, 'ipc_france.csv'))
print(f"Brut : {df.shape}")

df_ipc = nettoyer_insee(df)
df_ipc = df_ipc.rename(columns={'valeur': 'indice_prix'})
df_ipc['mois'] = df_ipc['periode'].str[5:7].astype(int)

# Calcul inflation 12 mois
df_ipc = df_ipc.sort_values('periode')
df_ipc['inflation_12m'] = df_ipc['indice_prix'].pct_change(12) * 100

print(f"Nettoyé : {df_ipc.shape}")
df_ipc.tail()

Brut : (431, 56)
Nettoyé : (431, 8)


,periode,indice_prix,description,zone,id_serie,annee,mois,inflation_12m
426,2025-07,121.72,Indice des prix à la consommation - Base 2015 ...,FM,1763866,2025,7,0.987306
427,2025-08,122.24,Indice des prix à la consommation - Base 2015 ...,FM,1763866,2025,8,0.874732
428,2025-09,121.05,Indice des prix à la consommation - Base 2015 ...,FM,1763866,2025,9,1.153171
429,2025-10,121.13,Indice des prix à la consommation - Base 2015 ...,FM,1763866,2025,10,0.933256
430,2025-11,120.89,Indice des prix à la consommation - Base 2015 ...,FM,1763866,2025,11,0.893006


## 6. Prix des logements

In [6]:
df = pd.read_csv(os.path.join(DATA_DIR, 'prix_logements_indices.csv'))
print(f"Brut : {df.shape}")

df_prix = nettoyer_insee(df)
df_prix = df_prix.rename(columns={'valeur': 'indice_prix'})
df_prix['trimestre'] = df_prix['periode'].str[-1].astype(int)

print(f"Nettoyé : {df_prix.shape}")
df_prix.head()

Brut : (15296, 50)
Nettoyé : (15292, 7)


,periode,indice_prix,description,zone,id_serie,annee,trimestre
0,2000-Q1,47.7,Indice des prix des logements (neufs et ancien...,FM,10001868,2000,1
1,2000-Q2,49.1,Indice des prix des logements (neufs et ancien...,FM,10001868,2000,2
2,2000-Q3,50.8,Indice des prix des logements (neufs et ancien...,FM,10001868,2000,3
3,2000-Q4,50.6,Indice des prix des logements (neufs et ancien...,FM,10001868,2000,4
4,2001-Q1,51.2,Indice des prix des logements (neufs et ancien...,FM,10001868,2001,1


## 7. Chômage national

In [7]:
df = pd.read_csv(os.path.join(DATA_DIR, 'chomage_series.csv'))
print(f"Brut : {df.shape}")

df_chomage_nat = nettoyer_insee(df)
df_chomage_nat = df_chomage_nat.rename(columns={'valeur': 'taux_chomage'})

print(f"Nettoyé : {df_chomage_nat.shape}")
df_chomage_nat.head()

Brut : (19130, 50)
Nettoyé : (19130, 6)


,periode,taux_chomage,description,zone,id_serie,annee
0,1975-Q1,644.0,Chômeurs au sens du BIT - Ensemble (en millier...,FM,1688358,1975
1,1975-Q2,718.0,Chômeurs au sens du BIT - Ensemble (en millier...,FM,1688358,1975
2,1975-Q3,790.0,Chômeurs au sens du BIT - Ensemble (en millier...,FM,1688358,1975
3,1975-Q4,812.0,Chômeurs au sens du BIT - Ensemble (en millier...,FM,1688358,1975
4,1976-Q1,822.0,Chômeurs au sens du BIT - Ensemble (en millier...,FM,1688358,1976


## 8. Chômage par département

In [8]:
df = pd.read_csv(os.path.join(DATA_DIR, 'chomage_departements_complet.csv'))
print(f"Brut : {df.shape}")

df_chomage_dept = nettoyer_insee(df)
df_chomage_dept = df_chomage_dept.rename(columns={'valeur': 'taux_chomage'})

print(f"Nettoyé : {df_chomage_dept.shape}")
df_chomage_dept.head()

Brut : (16988, 43)
Nettoyé : (16988, 6)


,periode,taux_chomage,description,zone,id_serie,annee
0,1982-Q1,5.2,Taux de chômage localisé par département - Ave...,D12,1515862,1982
1,1982-Q2,5.3,Taux de chômage localisé par département - Ave...,D12,1515862,1982
2,1982-Q3,5.5,Taux de chômage localisé par département - Ave...,D12,1515862,1982
3,1982-Q4,5.5,Taux de chômage localisé par département - Ave...,D12,1515862,1982
4,1983-Q1,5.6,Taux de chômage localisé par département - Ave...,D12,1515862,1983


## 9. Chômage par région

In [9]:
df = pd.read_csv(os.path.join(DATA_DIR, 'chomage_regions.csv'))
print(f"Brut : {df.shape}")

df_chomage_reg = nettoyer_insee(df)
df_chomage_reg = df_chomage_reg.rename(columns={'valeur': 'taux_chomage'})

print(f"Nettoyé : {df_chomage_reg.shape}")
df_chomage_reg.head()

Brut : (2100, 42)
Nettoyé : (2100, 6)


,periode,taux_chomage,description,zone,id_serie,annee
0,1982-Q1,7.0,Taux de chômage localisé par département - Hau...,D70,1515935,1982
1,1982-Q2,7.2,Taux de chômage localisé par département - Hau...,D70,1515935,1982
2,1982-Q3,7.2,Taux de chômage localisé par département - Hau...,D70,1515935,1982
3,1982-Q4,7.0,Taux de chômage localisé par département - Hau...,D70,1515935,1982
4,1983-Q1,6.7,Taux de chômage localisé par département - Hau...,D70,1515935,1983


## 10. Population départements

In [10]:
df = pd.read_csv(os.path.join(DATA_DIR, 'population_departements.csv'))
print(f"Brut : {df.shape}")
print(f"Colonnes brutes : {df.columns.tolist()}")

df_pop = nettoyer_recensement(df, nom_valeur='nb_logements')

print(f"\nNettoyé : {df_pop.shape}")
print(f"Colonnes : {df_pop.columns.tolist()}")
df_pop.head()

Brut : (2003, 10)
Colonnes brutes : ['CODEGEO', 'NIVGEO', 'UNIT_label_fr', 'UNIT', 'STOCD', 'OBS_VALUE', 'STOCD_label', 'DATASET_VERSION', 'DATASET_NAME', 'DATA_DATE']

Nettoyé : (1800, 6)
Colonnes : ['code_dept', 'nb_logements', 'annee', 'unite', 'STOCD_label', 'UNIT']


,code_dept,nb_logements,annee,unite,STOCD_label,UNIT
0,01,274538.375368,2018,Nombre de logements,Statut d'occupation du logement,NBLOG
1,01,171307.121971,2018,Nombre de logements,Statut d'occupation du logement,NBLOG
2,01,54028.486208,2018,Nombre de logements,Statut d'occupation du logement,NBLOG
3,01,39004.585669,2018,Nombre de logements,Statut d'occupation du logement,NBLOG
4,01,5205.883730,2018,Nombre de logements,Statut d'occupation du logement,NBLOG


## 11. Population séries temporelles

In [11]:
df = pd.read_csv(os.path.join(DATA_DIR, 'population_departements_series.csv'))
print(f"Brut : {df.shape}")

df_pop_series = nettoyer_insee(df)
df_pop_series = df_pop_series.rename(columns={'valeur': 'population'})

print(f"Nettoyé : {df_pop_series.shape}")
df_pop_series.head()

Brut : (9687, 47)
Nettoyé : (9687, 6)


,periode,population,description,zone,id_serie,annee
0,1975,34.5,Estimations de population - Part des 0-24 ans ...,D24,1741255,1975
1,1976,34.2,Estimations de population - Part des 0-24 ans ...,D24,1741255,1976
2,1977,33.8,Estimations de population - Part des 0-24 ans ...,D24,1741255,1977
3,1978,33.4,Estimations de population - Part des 0-24 ans ...,D24,1741255,1978
4,1979,33.0,Estimations de population - Part des 0-24 ans ...,D24,1741255,1979


## 12. Population France

In [12]:
df = pd.read_csv(os.path.join(DATA_DIR, 'population_serie_france.csv'))
print(f"Brut : {df.shape}")

df_pop_fr = nettoyer_insee(df)
df_pop_fr = df_pop_fr.rename(columns={'valeur': 'population'})

print(f"Nettoyé : {df_pop_fr.shape}")
df_pop_fr.head()

Brut : (383, 50)
Nettoyé : (383, 6)


,periode,population,description,zone,id_serie,annee
0,1994-01,59070,Démographie - Population au début du mois - Fr...,FE,1641607,1994
1,1994-02,59078,Démographie - Population au début du mois - Fr...,FE,1641607,1994
2,1994-03,59090,Démographie - Population au début du mois - Fr...,FE,1641607,1994
3,1994-04,59105,Démographie - Population au début du mois - Fr...,FE,1641607,1994
4,1994-05,59122,Démographie - Population au début du mois - Fr...,FE,1641607,1994


## 13. Revenus départements

In [13]:
df = pd.read_csv(os.path.join(DATA_DIR, 'revenus_departements.csv'))
print(f"Brut : {df.shape}")
print(f"Colonnes brutes : {df.columns.tolist()}")

df_revenus = nettoyer_recensement(df, nom_valeur='revenu')

print(f"\nNettoyé : {df_revenus.shape}")
print(f"Colonnes : {df_revenus.columns.tolist()}")
df_revenus.head()

Brut : (1770, 12)
Colonnes brutes : ['CODEGEO', 'NIVGEO', 'UNIT_label_fr', 'UNIT', 'INDICS_FILO_DISP', 'OBS_VALUE', 'INDICS_FILO_DISP_label', 'DATASET_VERSION', 'DATASET_NAME', 'DATA_DATE', 'INDICS_FILO_DISP_DET', 'INDICS_FILO_DISP_DET_label']

Nettoyé : (1764, 5)
Colonnes : ['code_dept', 'revenu', 'annee', 'unite', 'UNIT']


,code_dept,revenu,annee,unite,UNIT
0,01,258018.0,2018,Nombre de ménages fiscaux,NBMEN
1,01,618263.0,2018,Nombre de personnes fiscales,NBPERS
2,01,23420.0,2018,Médiane,MEDIANE
3,02,221370.0,2018,Nombre de ménages fiscaux,NBMEN
4,02,512838.0,2018,Nombre de personnes fiscales,NBPERS


## 14. Revenus des ménages

In [14]:
df = pd.read_csv(os.path.join(DATA_DIR, 'revenus_menages_departements.csv'))
print(f"Brut : {df.shape}")

df_rev_menages = nettoyer_insee(df)
df_rev_menages = df_rev_menages.rename(columns={'valeur': 'revenu'})

print(f"Nettoyé : {df_rev_menages.shape}")
df_rev_menages.head()

Brut : (1940, 38)
Nettoyé : (1940, 6)


,periode,revenu,description,zone,id_serie,annee
0,2012,18974.0,Revenu disponible par unité de consommation : ...,D18,1741916,2012
1,2013,19441.0,Revenu disponible par unité de consommation : ...,D18,1741916,2013
2,2014,19617.0,Revenu disponible par unité de consommation : ...,D18,1741916,2014
3,2015,19772.0,Revenu disponible par unité de consommation : ...,D18,1741916,2015
4,2012,18880.0,Revenu disponible par unité de consommation : ...,D19,1741917,2012


## 15. Pauvreté

In [15]:
df = pd.read_csv(os.path.join(DATA_DIR, 'pauvrete_departements.csv'))
print(f"Brut : {df.shape}")

df_pauvrete = nettoyer_insee(df)
df_pauvrete = df_pauvrete.rename(columns={'valeur': 'taux_pauvrete'})

print(f"Nettoyé : {df_pauvrete.shape}")
df_pauvrete.head()

Brut : (2716, 41)
Nettoyé : (2716, 6)


,periode,taux_pauvrete,description,zone,id_serie,annee
0,2012,13.6,Taux de pauvreté : Moins de 30 ans - Vendée,D85,1742658,2012
1,2013,14.5,Taux de pauvreté : Moins de 30 ans - Vendée,D85,1742658,2013
2,2014,14.5,Taux de pauvreté : Moins de 30 ans - Vendée,D85,1742658,2014
3,2015,14.8,Taux de pauvreté : Moins de 30 ans - Vendée,D85,1742658,2015
4,2012,23.6,Taux de pauvreté : Moins de 30 ans - Vienne,D86,1742659,2012


## 16. Emploi

In [16]:
df = pd.read_csv(os.path.join(DATA_DIR, 'emploi_departements.csv'))
print(f"Brut : {df.shape}")
print(f"Colonnes brutes : {df.columns.tolist()[:10]}...")

df_emploi = nettoyer_recensement(df, nom_valeur='nb_etablissements')

print(f"\nNettoyé : {df_emploi.shape}")
print(f"Colonnes : {df_emploi.columns.tolist()}")
df_emploi.head()

Brut : (5402, 14)
Colonnes brutes : ['CODEGEO', 'NIVGEO', 'UNIT_label_fr', 'UNIT', 'NA17', 'OBS_VALUE', 'NA17_label', 'DATASET_VERSION', 'DATASET_NAME', 'DATA_DATE']...

Nettoyé : (5396, 6)
Colonnes : ['code_dept', 'nb_etablissements', 'annee', 'unite', 'NA17_label', 'UNIT']


,code_dept,nb_etablissements,annee,unite,NA17_label,UNIT
0,01,18607.0,2018,Nombre d'établissements,Activité économique en 17 postes (NA - A17),NBET
1,01,679.0,2018,Nombre d'établissements,Activité économique en 17 postes (NA - A17),NBET
2,01,444.0,2018,Nombre d'établissements,Activité économique en 17 postes (NA - A17),NBET
3,01,0.0,2018,Nombre d'établissements,Activité économique en 17 postes (NA - A17),NBET
4,01,145.0,2018,Nombre d'établissements,Activité économique en 17 postes (NA - A17),NBET


## 17. Entreprises

In [17]:
df = pd.read_csv(os.path.join(DATA_DIR, 'entreprises_departements.csv'))
print(f"Brut : {df.shape}")
print(f"Colonnes brutes : {df.columns.tolist()[:10]}...")

df_entreprises = nettoyer_recensement(df, nom_valeur='nb_creations')

print(f"\nNettoyé : {df_entreprises.shape}")
print(f"Colonnes : {df_entreprises.columns.tolist()}")
df_entreprises.head()

Brut : (3333, 12)
Colonnes brutes : ['CODEGEO', 'NIVGEO', 'UNIT_label_fr', 'UNIT', 'NA10_HORS_AZ', 'OBS_VALUE', 'NA10_HORS_AZ_label', 'DATASET_VERSION', 'DATASET_NAME', 'DATA_DATE']...

Nettoyé : (3328, 6)
Colonnes : ['code_dept', 'nb_creations', 'annee', 'unite', 'NA10_HORS_AZ_label', 'UNIT']


,code_dept,nb_creations,annee,unite,NA10_HORS_AZ_label,UNIT
0,01,7304,2020,Nombre de créations d'établissements,"Activité économique en 10 postes, hors AZ (NA ...",NBCRET
1,01,534,2020,Nombre de créations d'établissements,"Activité économique en 10 postes, hors AZ (NA ...",NBCRET
2,01,871,2020,Nombre de créations d'établissements,"Activité économique en 10 postes, hors AZ (NA ...",NBCRET
3,01,2002,2020,Nombre de créations d'établissements,"Activité économique en 10 postes, hors AZ (NA ...",NBCRET
4,01,256,2020,Nombre de créations d'établissements,"Activité économique en 10 postes, hors AZ (NA ...",NBCRET


---

## 18. Données complémentaires

Les sections suivantes traitent des données téléchargées manuellement ou extraites en complément.

### 18.1 FILOSOFI 2021 (Revenus et pauvreté par commune)

Données plus récentes téléchargées depuis l'INSEE. Ces fichiers complètent les données TCRED 2012-2015 avec des informations au niveau communal pour 2021.

In [18]:
# FILOSOFI 2021 - Revenus par commune
fichier_filo = os.path.join(DATA_DIR, 'FILO2021_DISP_COM.csv')

if os.path.exists(fichier_filo):
    df_filo = pd.read_csv(fichier_filo, sep=';', low_memory=False)
    print(f"FILOSOFI 2021 chargé : {df_filo.shape}")
    
    # Colonnes essentielles
    cols_revenus = ['CODGEO', 'NBMEN21', 'NBPERS21', 'Q221']  # Q221 = médiane
    df_revenus_2021 = df_filo[cols_revenus].copy()
    df_revenus_2021.columns = ['code_commune', 'nb_menages', 'nb_personnes', 'revenu_median']
    
    # Conversion des valeurs (les 's' = secret statistique -> NaN)
    for col in ['nb_menages', 'nb_personnes', 'revenu_median']:
        df_revenus_2021[col] = pd.to_numeric(df_revenus_2021[col], errors='coerce')
    
    # Extraire code département
    df_revenus_2021['code_dept'] = df_revenus_2021['code_commune'].str[:2]
    df_revenus_2021['annee'] = 2021
    
    # Supprimer les lignes sans revenu médian
    df_revenus_2021 = df_revenus_2021.dropna(subset=['revenu_median'])
    
    print(f"Nettoyé : {df_revenus_2021.shape}")
    print(f"Communes avec données : {len(df_revenus_2021)}")
    df_revenus_2021.head()
else:
    print("Fichier FILOSOFI 2021 non trouvé")
    df_revenus_2021 = None

FILOSOFI 2021 chargé : (34929, 732)
Nettoyé : (31325, 6)
Communes avec données : 31325


In [19]:
# FILOSOFI 2021 - Pauvreté par commune
fichier_pauvrete = os.path.join(DATA_DIR, 'FILO2021_DISP_PAUVRES_COM.csv')

if os.path.exists(fichier_pauvrete):
    df_pauv = pd.read_csv(fichier_pauvrete, sep=';', low_memory=False)
    print(f"Pauvreté 2021 chargé : {df_pauv.shape}")
    
    # TP6021 = taux de pauvreté à 60%
    cols_pauvrete = ['CODGEO', 'TP6021']
    df_pauvrete_2021 = df_pauv[cols_pauvrete].copy()
    df_pauvrete_2021.columns = ['code_commune', 'taux_pauvrete']
    
    # Conversion (virgule décimale + 's' = secret statistique -> NaN)
    df_pauvrete_2021['taux_pauvrete'] = df_pauvrete_2021['taux_pauvrete'].str.replace(',', '.', regex=False)
    df_pauvrete_2021['taux_pauvrete'] = pd.to_numeric(df_pauvrete_2021['taux_pauvrete'], errors='coerce')
    
    df_pauvrete_2021['code_dept'] = df_pauvrete_2021['code_commune'].str[:2]
    df_pauvrete_2021['annee'] = 2021
    
    df_pauvrete_2021 = df_pauvrete_2021.dropna(subset=['taux_pauvrete'])
    
    print(f"Nettoyé : {df_pauvrete_2021.shape}")
    print(f"Note : {100 - len(df_pauvrete_2021)/34929*100:.0f}% des communes sous secret statistique")
    df_pauvrete_2021.head()
else:
    print("Fichier pauvreté 2021 non trouvé")
    df_pauvrete_2021 = None

Pauvreté 2021 chargé : (34929, 141)
Nettoyé : (4396, 4)
Note : 87% des communes sous secret statistique


### 18.2 Construction de logements

In [20]:
# Construction de logements (permis et mises en chantier)
fichier_constr = os.path.join(DATA_DIR, 'construction_logements.csv')

if os.path.exists(fichier_constr):
    df = pd.read_csv(fichier_constr)
    print(f"Construction logements brut : {df.shape}")
    
    df_construction = nettoyer_insee(df)
    df_construction = df_construction.rename(columns={'valeur': 'nb_logements'})
    df_construction['mois'] = df_construction['periode'].str[5:7].astype(int)
    
    print(f"Nettoyé : {df_construction.shape}")
    print(f"Période : {df_construction['periode'].min()} → {df_construction['periode'].max()}")
    df_construction.head()
else:
    print("Fichier construction_logements.csv non trouvé")
    df_construction = None

Construction logements brut : (26000, 53)
Nettoyé : (26000, 7)
Période : 2000-01 → 2025-10


### 18.3 Parc de logements

In [21]:
# Parc de logements (total par catégorie)
fichier_parc = os.path.join(DATA_DIR, 'parc_logements.csv')

if os.path.exists(fichier_parc):
    df = pd.read_csv(fichier_parc)
    print(f"Parc logements brut : {df.shape}")
    
    df_parc = nettoyer_insee(df)
    df_parc = df_parc.rename(columns={'valeur': 'nb_logements'})
    
    print(f"Nettoyé : {df_parc.shape}")
    print(f"Période : {df_parc['annee'].min()} → {df_parc['annee'].max()}")
    df_parc.head()
else:
    print("Fichier parc_logements.csv non trouvé")
    df_parc = None

Parc logements brut : (1144, 41)
Nettoyé : (1144, 6)
Période : 1982 → 2025


---

## 19. Sauvegarde

In [22]:
# Dictionnaire des fichiers à sauvegarder
fichiers_clean = {
    # Macroéconomie
    'pib_clean.csv': df_pib,
    'ipc_clean.csv': df_ipc,
    'prix_logements_clean.csv': df_prix,
    
    # Chômage
    'chomage_national_clean.csv': df_chomage_nat,
    'chomage_departements_clean.csv': df_chomage_dept,
    'chomage_regions_clean.csv': df_chomage_reg,
    
    # Population
    'population_logements_clean.csv': df_pop,
    'population_series_clean.csv': df_pop_series,
    'population_france_clean.csv': df_pop_fr,
    
    # Revenus & Pauvreté
    'revenus_departements_clean.csv': df_revenus,
    'revenus_menages_clean.csv': df_rev_menages,
    'pauvrete_clean.csv': df_pauvrete,
    
    # Emploi & Entreprises
    'emploi_clean.csv': df_emploi,
    'entreprises_clean.csv': df_entreprises,
    
    # Construction & Parc de logements
    'construction_logements_clean.csv': df_construction,
    'parc_logements_clean.csv': df_parc,
    
    # FILOSOFI 2021 (revenus et pauvreté par commune)
    'revenus_communes_2021_clean.csv': df_revenus_2021,
    'pauvrete_communes_2021_clean.csv': df_pauvrete_2021
}

print("Sauvegarde des fichiers nettoyés...\n")
for nom, df in fichiers_clean.items():
    if df is not None:
        chemin = os.path.join(CLEAN_DIR, nom)
        df.to_csv(chemin, index=False)
        print(f"  {nom:45} {len(df):>6} lignes, {df.shape[1]} colonnes")
    else:
        print(f"  {nom:45} [non disponible]")

n_ok = sum(1 for df in fichiers_clean.values() if df is not None)
print(f"\n{n_ok} fichiers sauvegardés dans {CLEAN_DIR}")

Sauvegarde des fichiers nettoyés...

  pib_clean.csv                                  20435 lignes, 7 colonnes
  ipc_clean.csv                                    431 lignes, 8 colonnes
  prix_logements_clean.csv                       15292 lignes, 7 colonnes
  chomage_national_clean.csv                     19130 lignes, 6 colonnes
  chomage_departements_clean.csv                 16988 lignes, 6 colonnes
  chomage_regions_clean.csv                       2100 lignes, 6 colonnes
  population_logements_clean.csv                  1800 lignes, 6 colonnes
  population_series_clean.csv                     9687 lignes, 6 colonnes
  population_france_clean.csv                      383 lignes, 6 colonnes
  revenus_departements_clean.csv                  1764 lignes, 5 colonnes
  revenus_menages_clean.csv                       1940 lignes, 6 colonnes
  pauvrete_clean.csv                              2716 lignes, 6 colonnes
  emploi_clean.csv                                5396 lignes, 6 colonnes
 

## 20. Récapitulatif

In [23]:
print("=" * 65)
print("DONNÉES NETTOYÉES")
print("=" * 65)

total = 0
for f in sorted(os.listdir(CLEAN_DIR)):
    if f.endswith('.csv'):
        chemin = os.path.join(CLEAN_DIR, f)
        df = pd.read_csv(chemin)
        size = os.path.getsize(chemin) / 1024
        total += len(df)
        print(f"{f:40} {len(df):>7} lignes  {size:>7.1f} KB")

print("=" * 65)
print(f"Total : {total:,} lignes")

DONNÉES NETTOYÉES
chomage_departements_clean.csv             16988 lignes   1418.7 KB
chomage_national_clean.csv                 19130 lignes   2768.3 KB
chomage_regions_clean.csv                   2100 lignes    175.5 KB
construction_logements_clean.csv           26000 lignes   4090.0 KB
emploi_clean.csv                            5396 lignes    453.2 KB
entreprises_clean.csv                       3328 lignes    324.8 KB
ipc_clean.csv                                431 lignes     66.5 KB
parc_logements_clean.csv                    1144 lignes    170.3 KB
pauvrete_clean.csv                          2716 lignes    191.7 KB
pauvrete_communes_2021_clean.csv            4396 lignes     80.3 KB
pib_clean.csv                              20435 lignes   2783.7 KB
population_france_clean.csv                  383 lignes     44.2 KB
population_logements_clean.csv              1800 lignes    144.2 KB
population_series_clean.csv                 9687 lignes    794.8 KB
prix_logements_clean.csv      

---

## Bilan

**18 fichiers nettoyés** dans `data/clean/` :

| Catégorie | Fichiers | Colonnes principales |
|-----------|----------|---------------------|
| Macroéconomie | pib, ipc, prix_logements | periode, valeur, zone, annee |
| Chômage | national, départements, régions | periode, taux_chomage, zone, annee |
| Population | logements, séries, france | code_dept/zone, valeur, annee |
| Revenus | départements, ménages, revenus_2021 | code_dept/zone, revenu, annee |
| Pauvreté | pauvrete, pauvrete_2021 | code_dept/zone, taux_pauvrete, annee |
| Emploi | emploi, entreprises | code_dept, nb_etablissements, annee |
| Logements | construction, parc | periode, nb_logements, annee |

### Traitements appliqués :
- Suppression des colonnes techniques INSEE (>50 colonnes réduites à 5-7)
- Renommage des variables pour plus de lisibilité
- Suppression des valeurs manquantes sur les colonnes clés
- Suppression des colonnes avec >50% de NaN
- Dédoublonnage des données
- Conversion des valeurs 's' (secret statistique) en NaN pour FILOSOFI 2021

### Sources des données :
- **API INSEE (pynsee)** : PIB, IPC, chômage, population, revenus, emploi, prix logements
- **Téléchargement manuel** : FILOSOFI 2021 (revenus et pauvreté par commune)

**Notebook suivant** : `04_analyse_exploratoire.ipynb`